### **imports**

In [16]:
import pandas as pd
import numpy as np
from astropy.io import ascii
from astropy.table import QTable, Table, Column
from astropy import units as u
import string
import os
import upsilon

### **making a table of all of the stars in the txt file**

In [17]:
column1 = []
column2 = []
column3 = []
column4 = []
column5 = []

with open('rrlyr_vsx_clean2.txt', 'r') as file:
    for line in file:
        components = line.split(' ', 4)
    
        try:
            magnitude = float(components[3])  # Try to convert the 4th component to float
        except ValueError:
            continue

        if magnitude < 15.5:
            column1.append(components[0])
            column2.append(float(components[1]))
            column3.append(float(components[2]))
            column4.append(magnitude)
            column5.append(components[4].strip())

final_table = QTable([column1, column2, column3, column4, column5], names = ("Star Type", "RA", "Dec", "Magnitude", "Other Info"))

In [18]:
final_table

Star Type,RA,Dec,Magnitude,Other Info
str24,float64,float64,float64,str29
RRAB,0.01521,35.36286,12.43,GM And
RRAB,0.01669,18.40698,15.171,CSS_J000004.0+182425
RRAB,0.03171,34.67397,14.95,SERIV 27
RRAB,0.06821,-72.77875,15.1,BP Tuc
RRAB,0.099,36.3287,14.46,NSVS 6313844
RRAB,0.1338,22.99388,14.671,CSS_J000032.1+225937
RRAB,0.13423,-54.29512,15.26,ASASSN-V J000032.21-541742.4
RRAB,0.14829,26.66375,12.96,GV Peg
RRC,0.15377,-60.98439,15.18,ASASSN-V J000036.92-605903.9


### **importing the asassn client**

In [19]:
from pyasassn.client import SkyPatrolClient	
client = SkyPatrolClient()

Welcome to ASAS-SN Skypatrol!

Current Deployment Version: 0.6.17 (26 JAN 2024)
Please upgrade your client if not up to date.



### **downloading a csv file with the data from the star at a specified index**

In [20]:
def classification_model(path, ra, dec, magnitude1, vsx_id, asassn_id):
  # Load a classification model.
  # (takes time, do only once if processing multiple lcs)
  rf_model = upsilon.load_rf_model()


  ############# Read input lc data #############
  # Read the input lightcurve file
  f = open(path, 'r')
  data = np.loadtxt(f, dtype={'names': ('JD', 'mag', 'err'),'formats': ('float64', 'float64', 'float64')})
  f.close()

  # Split it into three arrays
  date=data['JD']
  mag=data['mag']
  err=data['err']

  # Sigma-clip the input lightcurve
  date, mag, err = upsilon.utils.sigma_clipping(date, mag, err,  threshold=3, iteration=1)

  ############# Extract lightcurve features #############
  e_features = upsilon.ExtractFeatures(date, mag, err)
  e_features.run()
  features = e_features.get_features()

  ############# Classify #############
  # Classify the light curve
  label, probability, flag = upsilon.predict(rf_model, features)

  # Print results

  output_file = open(path.replace('.txt', '_classification.txt'), 'w')
  output_file.write("\n\n######### Classification results #########\n")
  output_file.write("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)\n")
  output_file.write("class = " + str(label) + "\n")
  output_file.write("class_probability = " + str(probability) + "\n")
  output_file.write("position = " + str(ra) + " " + str(dec) + "\n")
  output_file.write("magnitude = " + str(magnitude1) + "\n")
  output_file.write("vsx_id = " + str(vsx_id) + "\n")
  output_file.write("asassn_id = " + str(asassn_id) + "\n")
  output_file.write("color_string =\n")
  output_file.write("visual_inspection_comments =\n")
  output_file.close()

  print("\n\n######### Classification results #########\n")
  print("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)")
  print("class = " + str(label) )
  print("class_probability = " + str(probability) )
  print("\n\n######### Lightcurve features #########\n")

  for feature in features:
    print( str(feature) + " = " + str(features[feature]) )
  #  print feature," = ",features[feature]

In [21]:
folder_path = "/Users/zoesurles/Desktop/variable_star_classification/downloaded_lightcurves/"

In [22]:
num_iterations = 1
empty_list = []
for i in range(3000, 4000):

    try:
        search = client.cone_search(ra_deg = final_table["RA"][i], dec_deg = final_table["Dec"][i], radius=0.0003, catalog='aavsovsx', download=True)
    except:
        print("\n ################################################## error in index", i, "\n")
        continue

    savecsv = search.save(save_dir = folder_path, file_format="csv")[1]
    empty_list.append(savecsv)

    new_file = pd.read_csv(savecsv, skiprows=1)

    g_filter = new_file[new_file['phot_filter'] == 'g']
    quality = g_filter[g_filter['quality'] == 'G']
    web_format = quality[["jd", "mag", "mag_err"]]
    web_format.to_csv(savecsv.replace(".csv", ".txt"), sep='\t', index=False, header = False)

    classification_model(savecsv.replace(".csv", ".txt"), final_table["RA"][i], final_table["Dec"][i], final_table["Magnitude"][i], final_table["Other Info"][i], new_file["asas_sn_id"][0])

Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.16902012560426147
cusum = 0.19803283815418346
eta = 2.4516116371181726
hl_amp_ratio = 0.6850163939057706
kurtosis = -0.6786881416008725
n_points = 642
period = 0.6365548297741913
period_SNR = 146.75735985436293
period_log10FAP = -72.75187263678062
period_uncertainty = 0.00014044542836449958
phase_cusum = 0.34637644321165106
phase_eta = 0.8119859788636044
phi21 = 3.7287004717602423
phi31 = 2.1011087945707496
quartile31 = 0.24628627954953686
r21 = 0.3513622926376686
r31 = 0.18641942230665062
shapiro_w = 0.9823648929595947
skewness = -0.09375898568046784
slope_per10 = -0.060121823898890404
slope_per90 = 0.0767765337059129
stetson_k = 0.7503212533563882
weighted_mean = 15.596570480323464
weighted_std = 0.14773183144893387
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = NonVar
class_probability = 0.43


######### Lightcurve features #########

amplitude = 0.026539545235439832
cusum = 0.2625887955730291
eta = 2.8847816823105292
hl_amp_ratio = 1.31285308521688
kurtosis = 0.08401268181238564
n_points = 674
period = 0.27402075469844384
period_SNR = 24.90685692580044
period_log10FAP = -7.271641182732784
period_uncertainty = 3.3659348830983316e-05
phase_cusum = 0.25095471755891446
phase_eta = 2.747038746194219
phi21 = 0.2805895552399992
phi31 = 1.0813431924567494
quartile31 = 0.07931082526715194
r21 = 0.12313349748303025
r31 = 0.19609814720310795
shapiro_w = 0.9736044406890869
skewness = 0.5794044710294868
slope_per10 = -0.03272045506358548
slope_per90 = 0.03886814511809653
stetson_k = 0.8599235293352578
weighted_mean = 11.110340938834115
weighted_std = 0.05158314182850741
Pulled 1 of 1


...



######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.20677096467276374
cusum = 0.12005360670756995
eta = 2.3810976216339124
hl_amp_ratio = 0.5749474893362839
kurtosis = -0.909794354082095
n_points = 1022
period = 0.6371132362752377
period_SNR = 228.04291414994185
period_log10FAP = -151.62034403975557
period_uncertainty = 0.00014086699968440808
phase_cusum = 0.29846682596083096
phase_eta = 0.3129951026145049
phi21 = 0.8130958939807513
phi31 = 2.030197335199513
quartile31 = 0.281188598672637
r21 = 0.398243104702762
r31 = 0.2122628453460571
shapiro_w = 0.9553868174552917
skewness = -0.37228756560731474
slope_per10 = -0.071740522549838
slope_per90 = 0.05875524849442726
stetson_k = 0.7870193177779816
weighted_mean = 15.089107252425254
weighted_std = 0.16486740730189506
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 0.99


######### Lightcurve features #########

amplitude = 0.2144574772898219
cusum = 0.12059677596454277
eta = 2.1129981207719304
hl_amp_ratio = 0.4612665243910071
kurtosis = 0.07790660580626696
n_points = 811
period = 0.5753642949604217
period_SNR = 183.73171490759518
period_log10FAP = -99.24323878167961
period_uncertainty = 0.0001150544546970611
phase_cusum = 0.28599252741380415
phase_eta = 0.5746987107478139
phi21 = 0.5680453025563834
phi31 = 1.6565633927408923
quartile31 = 0.27041374733621915
r21 = 0.38590482928114517
r31 = 0.21832923645524288
shapiro_w = 0.9457386136054993
skewness = -0.7929388745306183
slope_per10 = -0.049340293100986574
slope_per90 = 0.04141114271661019
stetson_k = 0.7036020559729764
weighted_mean = 14.215843968768953
weighted_std = 0.19148800855897438
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.19680906702163414
cusum = 0.17139508598950987
eta = 2.022142696541966
hl_amp_ratio = 0.6788781260289435
kurtosis = -0.2612351367589709
n_points = 1096
period = 0.5553530222937783
period_SNR = 158.1006007772834
period_log10FAP = -92.20697687440281
period_uncertainty = 0.00010774348951558821
phase_cusum = 0.26422185372097895
phase_eta = 0.9151724420553115
phi21 = 0.8094464533863368
phi31 = 1.6628837267766248
quartile31 = 0.3031602056743239
r21 = 0.45922414884920876
r31 = 0.21315428683632673
shapiro_w = 0.9771425724029541
skewness = -0.48082467475699037
slope_per10 = -0.022486058588038618
slope_per90 = 0.018631374909726606
stetson_k = 0.7287230078209448
weighted_mean = 15.174656161822988
weighted_std = 0.21237369101546558
